In [ ]:
!curl http://127.0.0.1:11434

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

MODEL_NAME = "mistral"  # Change to "phi3" or "gemma:2b" if needed

# Initialize Ollama LLM
llm = OllamaLLM(
    model=MODEL_NAME,
    keep_alive=-1,
    format="json",
)

In [ ]:
import time
from tqdm import tqdm

In [ ]:
def llm_response_simple(request_str):
    system_prompt = ""
    
    # Create a properly formatted prompt
    prompt = ChatPromptTemplate.from_messages([
        # ("system", system_prompt),
        ("human", request_str)
    ])
    
    # Format the final prompt before passing it to `llm.invoke()`
    formatted_prompt = prompt.format_messages()
    
    # Get response from LLM
    response = llm.invoke(formatted_prompt)
    
    # Print the response
    return response 

In [ ]:
requests = [
    "Какой рецепт вегетарианской пасты?",
    "Как приготовить блинчики?",
    "Что можно приготовить из куриного филе?",
    "Как сделать шоколадный торт?",
    "Как приготовить рис для суши?",
    "Как приготовить суп-пюре из тыквы?",
    "Что можно приготовить из картошки и сыра?",
    "Как сделать домашний хлеб?",
    "Какие десерты можно приготовить без муки?",
    "Как приготовить рис с курицей и овощами?",
    "Как сделать соус для пасты?",
    "Как приготовить морковные котлеты?",
    "Что приготовить на ужин за 30 минут?",
    "Как сделать домашнюю пиццу?",
    "Что можно приготовить из куриного филе, риса и брокколи?",
    "Как приготовить сладкий омлет?",
    "Как сделать домашнее мороженое без мороженицы?",
    "Что можно приготовить для обеда, если нет мяса?",
    "Как приготовить крем-суп из грибов?",
    "Как приготовить рыбу с картошкой в духовке?",
    "Что приготовить для завтрака, если есть яйца, авокадо и помидоры?",
    "Как приготовить куриные крылышки на гриле?",
    "Что приготовить на праздничный ужин?",
    "Как приготовить соус бешамель?",
    "Как сделать торт без яиц?"
]

In [ ]:
responses = []
for req in tqdm(requests):
    start = time.time()
    res = llm_response_simple(req)
    responses.append({
        "request": req,
        "time": time.time() - start,
        "reqponse": res,
        "desc": "llm_response_simple"
    })
    break

In [ ]:
responses[0]

In [ ]:
SYSTEM_PROMPT_SIMPLE = "Ты — кулинарный помощник"
SYSTEM_PROMPT = """Ты — кулинарный помощник, который рекомендует рецепты на основе запросов пользователей. "
            "Используй доступные ингредиенты и предпочтения пользователя, чтобы предложить лучший рецепт. "
            "Если у пользователя нет конкретных предпочтений, предложи что-то популярное или сезонное. "
            "Опиши рецепт кратко: укажи название, основные ингредиенты и способ приготовления. "
            "Если возможно, укажи калорийность и полезные свойства блюда. "
            "Отвечай только на русском языке."""

def llm_response_with_prompt(request_str, system_prompt):
    
    # Create a properly formatted prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", request_str)
    ])
    
    # Format the final prompt before passing it to `llm.invoke()`
    formatted_prompt = prompt.format_messages()
    
    # Get response from LLM
    response = llm.invoke(formatted_prompt)
    
    # Print the response
    return response 

In [ ]:
responses_simple_prompt = []
for req in tqdm(requests):
    start = time.time()
    res = llm_response_with_prompt(req, SYSTEM_PROMPT_SIMPLE)
    responses_simple_prompt.append({
        "request": req,
        "time": time.time() - start,
        "reqponse": res,
        "desc": "llm_response_with_prompt(req, SYSTEM_PROMPT_SIMPLE)"
    })

In [ ]:
responses_prompt = []
for req in tqdm(requests):
    start = time.time()
    res = llm_response_with_prompt(req, SYSTEM_PROMPT)
    responses_prompt.append({
        "request": req,
        "time": time.time() - start,
        "reqponse": res,
        "desc": "llm_response_with_prompt(req, SYSTEM_PROMPT)"
    })

In [ ]:
import json
with open("responses.json", "w") as outfile: 
    json.dump(responses, outfile)

with open("responses_prompt.json", "w") as outfile: 
    json.dump(responses_prompt, outfile)

with open("responses_simple_prompt.json", "w") as outfile: 
    json.dump(responses_simple_prompt, outfile)

### Process documents

In [ ]:
import pickle
with open('recipe_str.pickle', 'rb') as handle:
    recipe_str = pickle.load(handle)

In [ ]:
import random
recipe_str_small = random.sample(recipe_str, 10)

In [ ]:
len(recipe_str_small)

In [ ]:
import os
import uuid
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

def process_documents(docs_dir: str = "documents"):
    # Initialize embeddings and text splitter
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        is_separator_regex=False,
    )

    # Process PDF files
    pdf_files = [f for f in os.listdir(docs_dir) if f.endswith(".pdf")]
    print(pdf_files)
    if not pdf_files:
        raise ValueError(f"No PDF files found in {docs_dir}")

    all_docs = []
    for pdf_file in pdf_files:
        file_path = os.path.join(docs_dir, pdf_file)
        print(file_path)
        loader = PyPDFLoader(file_path)
        pages = loader.load()
        
        # Add metadata to each page
        for page_num, page in enumerate(pages, start=1):
            page.metadata.update({
                "source": pdf_file,
                "page_number": page_num,
                "chunk_id": str(uuid.uuid4())[:8]
            })

        # Split pages into chunks
        chunks = text_splitter.split_documents(pages)
        all_docs.extend(chunks)

    # Create/update vector store
    Chroma.from_documents(
        documents=all_docs,
        embedding=embeddings,
        persist_directory="chroma_db_example_1",
        collection_metadata={"hnsw:space": "cosine"},
        collection_name="main_collection"
    )

In [ ]:
example = process_documents()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema.document import Document


def get_text_chunks_langchain():
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    docs = []
    for n, text in enumerate(recipe_str_small):
        doc = Document(page_content=text)
        doc.metadata.update({
                "chunk_id": str(uuid.uuid4())[:8]
            })
        docs.append(doc)
        # chunks = text_splitter.split_text([doc])
        # docs.extend(chunks)
    return docs


docs = get_text_chunks_langchain()

In [ ]:
print(len(docs))

In [ ]:
chunks = text_splitter.split_documents(docs)
all_docs = []
all_docs.extend(chunks)

In [ ]:
len(all_docs)

In [ ]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")
Chroma.from_documents(
        documents=all_docs,
        embedding=embeddings,
        persist_directory="chroma_db_recipe",
        collection_metadata={"hnsw:space": "cosine"},
        collection_name="main_collection"
    )

In [ ]:
from retrieve import DocumentRetriever
import ollama
import regex as re

class QAPipeline:
    def __init__(self):
        self.retriever = DocumentRetriever()
        
    PROMPT_TEMPLATE = """Context information:
        {context}

        Using the context above and your general knowledge, answer this question:
        Question: {question}

        Format requirements:

        - If uncertain, say "The documents don't specify"""

    def parse_response(self, response: str) -> dict:
        """Extract thinking and answer components without <answer> tags"""
        # Extract thinking process
        think_match = re.search(r'<think>(.*?)</think>', response, re.DOTALL)
        
        # Get everything AFTER </think> as the answer
        answer_start = response.find('</think>') + len('</think>')
        answer = response[answer_start:].strip()
        
        return {
            "thinking": think_match.group(1).strip() if think_match else "",
            "answer": answer,
            "raw_response": response
        }

    def generate_answer(self, question: str, k: int = 5) -> dict:
        """Full QA workflow with enhanced output"""
        try:
            # Retrieve documents
            context_docs = self.retriever.query_documents(question, k=k)
            
            # Format context preserving full metadata
            context_str = "\n".join(
                f"[Document {idx+1}] {doc['source']} (Page {doc['page']}):\n{doc['text']}"
                for idx, doc in enumerate(context_docs)
            )
            
            # Generate response
            response = ollama.generate(
                model="deepseek-r1:latest",
                prompt=self.PROMPT_TEMPLATE.format(
                    context=context_str,
                    question=question
                )
            )
            
            # Parse components
            parsed = self.parse_response(response['response'])
            
            return {
                **parsed,
                "sources": [
                    {
                        # "source": doc["source"],
                        # "page": doc["page"],
                        "confidence": doc["score"],
                        # "full_text": doc["text"]
                    } for doc in context_docs
                ]
            }
            
        except Exception as e:
            return {
                "error": str(e),
                "thinking": "",
                "answer": "Failed to generate response",
                "sources": []
            }
